# **4. Xây dựng trình phát hiện phần mềm độc hại bằng phân tích tĩnh**

B1: Tạo list các mẫu và gán nhãn cho chúng

In [ ]:
import os
from os import listdir
directories_with_labels = [("/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples", 0), 
                           ("/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Malicious PE Samples", 1)]
list_of_samples = []
labels = []
for dataset_path, label in directories_with_labels:
  samples = [f for f in listdir(dataset_path)]
  for sample in samples:
    file_path = os.path.join(dataset_path, sample)
    list_of_samples.append(file_path)
    labels.append(label)

B2: Tạo list các mẫu và gán nhãn cho chúng

In [ ]:
from sklearn.model_selection import train_test_split
samples_train, samples_test, labels_train, labels_test = train_test_split(list_of_samples, labels, test_size=0.3, stratify=labels, random_state=11)

B3: Các hàm lấy thuộc tính

In [ ]:
pip install collection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pefile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 5.9 MB/s eta 0:00:00


In [ ]:
def read_file(file_path):
  with open(file_path, "rb") as bin_file:
    data = bin_file.read()
    return data
def byte_seq_to_Ngrams(byte_seq, N_par):
  Ngrams_par = ngrams(byte_seq, N_par)
  return list(Ngrams_par)
def bin_file_to_Ngrams_count(file_path, N_par):
  file_seq = read_file(file_path)
  file_Ngrams = byte_seq_to_Ngrams(file_seq, N_par)
  return collections.Counter(file_Ngrams)
def get_Ngrams_features_from_samples(sample, K1_most_freq_Ngrams_list):
  K1 = len(K1_most_freq_Ngrams_list)
  feature_vector = K1 * [0]
  file_Ngrams = bin_file_to_Ngrams_count(sample, N)
  for i in range(K1):
    feature_vector[i] = file_Ngrams[K1_most_freq_Ngrams_list[i]]
  return feature_vector
def preprocess_imports(list_of_DLLs):
  """ Normalize the name of the imports of a PE file. """
  temp = [x.decode().split(".")[0].lower() for x in list_of_DLLs] # View the transforming of below example
  return " ".join(temp)
def get_imports(pe):
  """ Get a list of the imports of a PE file """
  list_of_imports = []
  for entry in pe.DIRECTORY_ENTRY_IMPORT:
    list_of_imports.append(entry.dll)
  return preprocess_imports(list_of_imports)
def get_section_names(pe):
  """ Get a list of the section names of a PE file """
  list_of_sections = []
  for sect in pe.sections:
    normalized_name = sect.Name.decode().replace("\x00", "").lower()
    list_of_sections.append(normalized_name)
  return "".join(list_of_sections)

B4: Chọn 100 thuộc tính phổ biến với 2-grams

In [ ]:
N = 2
Ngram_counts_all = collections.Counter([])
for sample in samples_train:
  Ngram_counts_all += bin_file_to_Ngrams_count(sample, N)
K1 = 100
K1_most_frequent_Ngrams = Ngram_counts_all.most_common(K1)
K1_most_frequent_Ngrams_list = [x[0] for x in K1_most_frequent_Ngrams]

B5: Trích xuất số lượng N-grams count, section names, imports và số lượng sections
của mỗi mẫu trong train-test

In [ ]:
imports_corpus_train = []
num_sections_train = []
section_names_train = []
Ngram_features_list_train = []
y_train = []
for i in range(len(samples_train)):
  sample = samples_train[i]
  try:
    NGram_features = get_Ngrams_features_from_samples(sample,K1_most_frequent_Ngrams_list)
    pe = pefile.PE(sample)
    imports = get_imports(pe)
    n_sections = len(pe.sections)
    sec_names = get_section_names(pe)
    imports_corpus_train.append(imports)
    num_sections_train.append(n_sections)
    section_names_train.append(sec_names)
    Ngram_features_list_train.append(NGram_features)
    y_train.append(labels_train[i])
  except Exception as e:
    print(sample + ":")
    print(e)

/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CExecSvc.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/ADSchemaAnalyzer.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/c2wtshost.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/BootExpCfg.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/dplaysvr.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/evntcmd.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/hvsirdpclient.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/iissetup.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Ben

B6: Sử dụng hàm băm tfidf để chuyển imports, section names từ văn bản thành dạng
số

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
imports_featurizer = Pipeline(
[
("vect", HashingVectorizer(input="content", ngram_range=(1,2))),
("tfidf", TfidfTransformer(use_idf=True,)),
]
)
section_names_featurizer = Pipeline(
[
("vect", HashingVectorizer(input="content", ngram_range=(1, 2))),
("tfidf", TfidfTransformer(use_idf=True,)),
]
)
imports_corpus_train_transformed = imports_featurizer.fit_transform(imports_corpus_train)
section_names_train_transformed = section_names_featurizer.fit_transform(section_names_train)

B7: Kết hợp các vector thuộc tính thành 1 mảng

In [ ]:
from scipy.sparse import hstack, csr_matrix
X_train = hstack(
[
  Ngram_features_list_train,
  imports_corpus_train_transformed,
  section_names_train_transformed,
  csr_matrix(num_sections_train).transpose(),
]
)

B8: Ta huấn luyên bằng phân loại Random Forest cho tập train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

B9: Thu thập các thuộc tính của tập test, giống như tập huấn luyện

In [ ]:
imports_corpus_test = []
num_sections_test = []
section_names_test = []
Ngram_features_list_test = []
y_test = []
for i in range(len(samples_test)):
  file = samples_test[i]
  try:
    NGram_features = get_Ngrams_features_from_samples(sample, K1_most_frequent_Ngrams_list)
    pe = pefile.PE(file)
    imports = get_imports(pe)
    n_sections = len(pe.sections)
    sec_names = get_section_names(pe)
    imports_corpus_test.append(imports)
    num_sections_test.append(n_sections)
    section_names_test.append(sec_names)
    Ngram_features_list_test.append(NGram_features)
    y_test.append(labels_test[i])
  except Exception as e:
    print(sample + ":")
    print(e)

/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content/drive/MyDrive/Machine Learning/Lab 2/Samples/Benign PE Samples/CompMgmtLauncher.exe:
'DOS Header magic not found.'
/content

In [ ]:
y_test

B10. Ta chuyển đổi vector từ thuộc tính test, và kiểm tra kết quả của trình phân loại

In [ ]:
import_corpus_test_transformed = imports_featurizer.transform(imports_corpus_test)
sect_names_test_transformed = imports_featurizer.transform(section_names_test)
X_test = hstack(
    [
        Ngram_features_list_test,
        import_corpus_test_transformed,
        sect_names_test_transformed, 
        csr_matrix(num_sections_test).transpose()
    ]
)

In [ ]:
print("Kết quả của trình phân loại: ")
print(clf.score(X_test, y_test))

Kết quả của trình phân loại: 
0.9801980198019802
